# Telangana RERA Scraper

1. Open the website ™ (and import stuff)

In [17]:
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import platform

browser: WebDriver

if platform == 'Darwin':
	browser = webdriver.Safari()
else:
    browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")

browser.get("http://rerait.telangana.gov.in/SearchList/Search")
page_size = browser.find_element_by_id('PageSize')

# Uncomment this when not in development; it'll use 100 elements per page instead of the default 10
# page_size.send_keys("0")

counter: int = 1

2. Scrape the 100 entries on the page

In [29]:
from selenium.webdriver.remote.webelement import WebElement

row_xpath = '//*[@id="gridview"]/div[1]/div/table/tbody/tr'

rows: list[WebElement] = browser.find_elements_by_xpath(row_xpath)
for row_i, row in enumerate(rows):
	cells: list[WebElement] = row.find_elements_by_xpath('{}[{}]/td'.format(row_xpath, row_i + 1))
	# Intentionally using camelCase instead of snake_case as the schema uses camelCase
	projectName = cells[1].get_attribute('innerHTML')
	promoterName = cells[2].get_attribute('innerHTML')
	lastModifiedDate = cells[3].get_attribute('innerHTML')
	# TODO: write data to a file

counter += 1 # Use this in the file name

SKV S ANANDA VILAS KACHAM  RAJESHWAR 31/08/2018 00:00:00
MY HOME KRISHE MY Home Constructions PVT. Ltd. 03/12/2021 00:00:00
RAJAPUSHPA ETERNA RAJAPUSHPA PROPERTIES PVT. LTD. 09/12/2021 00:00:00
SMR VINAY METRO MALL SMR BUILDERS PRIVATE LIMITED 03/12/2021 00:00:00
MYSCAPE-ISLE OF SKY MYSCAPE PROPERTIES PRIVATE LIMITED 02/12/2021 00:00:00
PRANEETH PRANAV ZENITH PRANEETH PROJECTS 18/11/2018 00:00:00
APURUPA JAGAPATI HEIGHTS APURUPA SHELTERS 10/11/2021 00:00:00
SILVER OAK VILLAS Silver Oak Villas LLP 17/05/2021 00:00:00
INDU ANNEXE PROJCT M/S CHIDCO PRIVATE LIMITED 28/06/2019 00:00:00
R.V.SOMWRITA R.V. NIRMAAN PVT.LTD., 31/01/2022 00:00:00
